In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df_blip = pd.read_csv('blip_captions.csv')

In [ ]:
df_load = pd.read_csv("40_classes_eeg_and_caption.csv")

In [ ]:
df_load.head()

,Unnamed: 0,caption,conditioning_image,eeg_embedding,caption_embedding
0,0,"image of a German shepherd, German shepherd do...",[[-1.0042979 -0.9780193 -1.0273838 ... -0.9...,"[0.07539713382720947, 1.9535454511642456, -0.4...",[[ 4.64860871e-02 1.42486207e-02 -3.59204337e...
1,1,image of a Egyptian cat,[[-0.9970702 -1.0927153 -1.0839258 ... -1.0...,"[0.24511180818080902, 1.4279429912567139, -1.0...",[[ 8.22394527e-03 5.36666401e-02 -1.19591681e...
2,2,"image of a lycaenid, lycaenid butterfly",[[-1.0036012 -1.0134891 -1.0905789 ... -0.9...,"[0.17774555087089539, 2.2378880977630615, -1.1...",[[ 3.25377285e-02 2.87974719e-02 3.87954563e...
3,3,image of a sorrel,[[-0.99896103 -0.99309397 -0.9720703 ... -1.0...,"[0.08839258551597595, 2.268357276916504, -0.75...",[[ 3.23821492e-02 4.56233770e-02 -2.87936740e...
4,4,"image of a capuchin, ringtail, Cebus capucinus",[[-0.99743646 -0.99578846 -0.9627674 ... -1.0...,"[0.023998700082302094, 2.328174591064453, -0.6...",[[ 0.05938439 0.00615933 -0.00307909 0.02962...


In [ ]:
# I want to appy USE on blip_captions
import tensorflow_hub as hub
import tensorflow as tf

# ✅ Load Google's Universal Sentence Encoder
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")


def get_embeddings(texts):
    return use_model(texts).numpy()


# Example:
sample_texts = ["This is a test sentence.",
                "Sexist comments should be flagged."]
embeddings = get_embeddings(sample_texts)

print(embeddings.shape)

(2, 512)


In [ ]:
df_load["caption_embedding"] = df_load["caption"].apply(lambda x: get_embeddings([x]))

In [ ]:
df_load = df_load.drop(columns=["Unnamed: 0"])

In [ ]:
import pandas as pd
import numpy as np
df_train = pd.DataFrame({ # Changed Dataframe to DataFrame
    "caption_embedding": df_load["caption_embedding"],
    "eeg_embedding": df_load["eeg_embedding"]
})

In [ ]:
# df_train["eeg_embedding"] = df_train["eeg_embedding"].str.split(" ")

In [ ]:
# reshape
# df_train["eeg_embedding"] = df_train["eeg_embedding"].apply(lambda x: np.array(x).reshape(1, -1))

In [ ]:
import pandas as pd
import numpy as np
import ast

# Assuming df_train is your DataFrame
# ... (other code) ...

# Convert the string representation to a list of floats
df_train["eeg_embedding"] = df_train["eeg_embedding"].apply(lambda x: [float(i) for i in ast.literal_eval(x)])

# Convert the list of floats to a NumPy array
df_train["eeg_embedding"] = df_train["eeg_embedding"].apply(lambda x: np.array(x).reshape(1, -1))

# Now you can proceed with further processing
# ... (rest of your code) ...

In [ ]:
df_train["caption_embedding"].iloc[0].shape

(1, 512)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# 📝 Load Data (Assuming df_train is your main DataFrame)
eeg_embeddings = np.array([np.array(x).flatten() for x in df_train["eeg_embedding"]])
caption_embeddings = np.array([np.array(x).flatten() for x in df_train["caption_embedding"]])

# Normalize the embeddings
eeg_embeddings = (eeg_embeddings - eeg_embeddings.mean(axis=0)) / eeg_embeddings.std(axis=0)
caption_embeddings = (caption_embeddings - caption_embeddings.mean(axis=0)) / caption_embeddings.std(axis=0)

# Final shape checks
print(f"EEG Embeddings Shape: {eeg_embeddings.shape}")          # Expected: [N, 512]
print(f"Caption Embeddings Shape: {caption_embeddings.shape}")  # Expected: [N, 512]

# 🧠 Enhanced Decoder Model
class EEGToBLIPDecoder(nn.Module):
    def __init__(self, input_dim=512, output_dim=512):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, output_dim)
        )
        self.attention = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Apply attention before the main model
        weights = self.attention(x)  # [N, 1]
        x = x * weights  # Weighted input
        return self.model(x)

# Initialize model
model = EEGToBLIPDecoder(input_dim=512, output_dim=512)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 🔄 Prepare Data for Training
X_train, X_val, y_train, y_val = train_test_split(eeg_embeddings, caption_embeddings, test_size=0.1, random_state=42)

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val = torch.tensor(y_val, dtype=torch.float32)

print(f"Training Data Shape: {X_train.shape}")  # [N_train, 512]
print(f"Validation Data Shape: {X_val.shape}")  # [N_val, 512]

# 🚀 Training Loop
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-2)
criterion_mse = nn.MSELoss()
criterion_cos = nn.CosineEmbeddingLoss()

epochs = 100
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

for epoch in range(epochs):
    # Training step
    model.train()
    optimizer.zero_grad()

    preds = model(X_train)
    mse_loss = criterion_mse(preds, y_train)
    cos_loss = criterion_cos(preds, y_train, torch.ones(preds.size(0)).to(device))

    # Combined loss (weighted)
    train_loss = mse_loss + 0.5 * cos_loss
    train_loss.backward()
    optimizer.step()
    scheduler.step()

    # Validation step
    model.eval()
    with torch.no_grad():
        val_preds = model(X_val)
        val_mse_loss = criterion_mse(val_preds, y_val)
        val_cos_loss = criterion_cos(val_preds, y_val, torch.ones(val_preds.size(0)).to(device))
        val_loss = val_mse_loss + 0.5 * val_cos_loss

    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss.item():.4f} | Val Loss: {val_loss.item():.4f}")

# 💾 Save the Trained Model
torch.save(model.state_dict(), "eeg_to_blip_decoder_enhanced.pth")
print("✅ Model saved as eeg_to_blip_decoder_enhanced.pth")


EEG Embeddings Shape: (40, 512)
Caption Embeddings Shape: (40, 512)
Training Data Shape: torch.Size([36, 512])
Validation Data Shape: torch.Size([4, 512])
Epoch 1/100 | Train Loss: 1.5155 | Val Loss: 1.4014
Epoch 2/100 | Train Loss: 1.5038 | Val Loss: 1.4063
Epoch 3/100 | Train Loss: 1.4946 | Val Loss: 1.4110
Epoch 4/100 | Train Loss: 1.4812 | Val Loss: 1.4160
Epoch 5/100 | Train Loss: 1.4753 | Val Loss: 1.4209
Epoch 6/100 | Train Loss: 1.4577 | Val Loss: 1.4259
Epoch 7/100 | Train Loss: 1.4537 | Val Loss: 1.4309
Epoch 8/100 | Train Loss: 1.4402 | Val Loss: 1.4360
Epoch 9/100 | Train Loss: 1.4310 | Val Loss: 1.4411
Epoch 10/100 | Train Loss: 1.4189 | Val Loss: 1.4457
Epoch 11/100 | Train Loss: 1.4081 | Val Loss: 1.4500
Epoch 12/100 | Train Loss: 1.3946 | Val Loss: 1.4541
Epoch 13/100 | Train Loss: 1.3923 | Val Loss: 1.4575
Epoch 14/100 | Train Loss: 1.3761 | Val Loss: 1.4604
Epoch 15/100 | Train Loss: 1.3660 | Val Loss: 1.4630
Epoch 16/100 | Train Loss: 1.3563 | Val Loss: 1.4651
Epoch 

## Model 2

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# 📝 Load Data (Assuming df_train is your main DataFrame)
eeg_embeddings = np.array([np.array(x).flatten() for x in df_train["eeg_embedding"]])
caption_embeddings = np.array([np.array(x).flatten() for x in df_train["caption_embedding"]])

# Final shape checks
print(f"EEG Embeddings Shape: {eeg_embeddings.shape}")          # Expected: [40, 512]
print(f"Caption Embeddings Shape: {caption_embeddings.shape}")  # Expected: [40, 512]

# 🧠 Define the Decoder Model
class EEGToBLIPDecoder(nn.Module):
    def __init__(self, input_dim=512, output_dim=512):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, output_dim)
        )

    def forward(self, x):
        return self.model(x)

# Initialize model
if eeg_embeddings.shape[1] == 512 and caption_embeddings.shape[1] == 512:
    model = EEGToBLIPDecoder(input_dim=512, output_dim=512)
else:
    raise ValueError("⚠️ Mismatched input/output dimensions. Make sure EEG embeddings have 512 features and Caption embeddings have 512 features.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 🔄 Prepare Data for Training
X_train, X_val, y_train, y_val = train_test_split(eeg_embeddings, caption_embeddings, test_size=0.1, random_state=42)

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val = torch.tensor(y_val, dtype=torch.float32)

print(f"Training Data Shape: {X_train.shape}")  # [N_train, 512]
print(f"Validation Data Shape: {X_val.shape}")  # [N_val, 512]

# 🚀 Training Loop
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()
epochs = 100

for epoch in range(epochs):
    # Training step
    model.train()
    optimizer.zero_grad()
    preds = model(X_train)
    train_loss = criterion(preds, y_train)
    train_loss.backward()
    optimizer.step()

    # Validation step
    model.eval()
    with torch.no_grad():
        val_preds = model(X_val)
        val_loss = criterion(val_preds, y_val)

    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss.item():.4f} | Val Loss: {val_loss.item():.4f}")

# 💾 Save the Trained Model
torch.save(model.state_dict(), "eeg_to_blip_decoder.pth")
print("✅ Model saved as eeg_to_blip_decoder.pth")


EEG Embeddings Shape: (40, 512)
Caption Embeddings Shape: (40, 512)
Training Data Shape: torch.Size([36, 512])
Validation Data Shape: torch.Size([4, 512])
Epoch 1/100 | Train Loss: 0.0097 | Val Loss: 0.0060
Epoch 2/100 | Train Loss: 0.0065 | Val Loss: 0.0042
Epoch 3/100 | Train Loss: 0.0045 | Val Loss: 0.0032
Epoch 4/100 | Train Loss: 0.0033 | Val Loss: 0.0026
Epoch 5/100 | Train Loss: 0.0027 | Val Loss: 0.0022
Epoch 6/100 | Train Loss: 0.0023 | Val Loss: 0.0020
Epoch 7/100 | Train Loss: 0.0021 | Val Loss: 0.0019
Epoch 8/100 | Train Loss: 0.0019 | Val Loss: 0.0018
Epoch 9/100 | Train Loss: 0.0018 | Val Loss: 0.0017
Epoch 10/100 | Train Loss: 0.0017 | Val Loss: 0.0016
Epoch 11/100 | Train Loss: 0.0017 | Val Loss: 0.0016
Epoch 12/100 | Train Loss: 0.0016 | Val Loss: 0.0015
Epoch 13/100 | Train Loss: 0.0015 | Val Loss: 0.0015
Epoch 14/100 | Train Loss: 0.0015 | Val Loss: 0.0015
Epoch 15/100 | Train Loss: 0.0014 | Val Loss: 0.0014
Epoch 16/100 | Train Loss: 0.0014 | Val Loss: 0.0014
Epoch 

In [ ]:
import torch
import numpy as np
import torch.nn as nn

# 🧠 Define the Decoder Model (must match the saved model architecture)
class EEGToBLIPDecoder(nn.Module):
    def __init__(self, input_dim=512, output_dim=512):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, output_dim)
        )
        self.attention = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        weights = self.attention(x)  # [N, 1]
        x = x * weights  # Weighted input
        return self.model(x)

# 🔄 Load the Model
model_path = "eeg_to_blip_decoder_enhanced.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EEGToBLIPDecoder(input_dim=512, output_dim=512).to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()
print("✅ Model loaded successfully.")

# 📝 Load a Sample EEG Embedding for Inference


✅ Model loaded successfully.


In [ ]:
df_ref = pd.read_csv("reference_captions_test.csv")

In [ ]:
reference_captions = np.unique(df_blip["blip_captions"].values)
ref_cap_2 = df_ref["captions"].values

In [ ]:
reference_captions = np.concatenate((reference_captions, ref_cap_2))

In [ ]:
reference_captions = np.unique(reference_captions)

In [ ]:
import sklearn
reference_embeddings = get_embeddings(reference_captions)
reference_embeddings = sklearn.preprocessing.normalize(reference_embeddings)

In [ ]:
# import numpy as np

# # Simulated predicted embedding (shape = [512])
# predicted_embedding = np.random.rand(512).astype(np.float32)

# # Normalize for cosine similarity (optional but typical for embeddings)
# predicted_embedding /= np.linalg.norm(predicted_embedding)


In [ ]:
# sample_eeg_embedding = np.random.randn(1, 512).astype(np.float32)  # Replace with real EEG data
sample_eeg_embedding = df_train["eeg_embedding"].iloc[12]  # Replace with real EEG data
sample_eeg_tensor = torch.tensor(sample_eeg_embedding, dtype=torch.float32).to(device) # Changed to torch.float32

# 🚀 Get the Predicted BLIP Embedding
with torch.no_grad():
    predicted_blip_embedding = model(sample_eeg_tensor).cpu().numpy()

print("Predicted BLIP Embedding Shape:", predicted_blip_embedding.shape)
print("Predicted BLIP Embedding (first 5 values):", predicted_blip_embedding[0][:5])

Predicted BLIP Embedding Shape: (1, 512)
Predicted BLIP Embedding (first 5 values): [-0.16533783  0.2950041  -0.08815338  0.01755551  0.2367802 ]


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_top_k_captions(predicted_embedding, reference_embeddings, reference_captions, k=5):
    sims = cosine_similarity(predicted_embedding.reshape(1, -1), reference_embeddings)[0]
    top_k_idx = np.argsort(sims)[-k:][::-1]  # Get indices of top-k similarities
    top_k_captions = [(reference_captions[i], sims[i]) for i in top_k_idx]
    return top_k_captions

# Example usage
top_captions = get_top_k_captions(predicted_blip_embedding, reference_embeddings, reference_captions, k=5)
# for i, (caption, score) in enumerate(top_captions):
#     print(f"{i+1}. {caption} (Similarity: {score:.3f})")


# # import random

# def get_random_caption_from_top_k(predicted_embedding, reference_embeddings, reference_captions, k=5):
#     top_k = get_top_k_captions(predicted_embedding, reference_embeddings, reference_captions, k)
#     return random.choice(top_k)

# # Example usage
# caption, score = get_random_caption_from_top_k(predicted_blip_embedding, reference_embeddings, reference_captions, k=5)
print(f"Retrieved Caption: {top_captions[0]}")


Retrieved Caption: ('a close up of a pair of running shoes on a box', np.float32(0.3160976))


## Stable Diffusion Image Generation

In [ ]:
!pip install diffusers

In [ ]:
from diffusers import DiffusionPipeline

pipe = DiffusionPipeline.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# pipe = pipe.to("cuda")

In [ ]:
prompt_seg = caption
image_seg = pipe(prompt_seg).images[0]

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(image)
plt.show()